In [42]:
import numpy as np
import pandas as pd
import cv2
import os
import json

from PIL import Image
# class 별 bbox 색

img_dir = './dataset'
output_dir = './best_output.csv'
with open("./dataset/train_aug_correct.json", "r") as json_file:
    aug_json = json.load(json_file)
# def createFolder(directory):
#     if not os.path.exists(directory):
#         os.makedirs(directory)
# createFolder('./dataset/train_seudo')

print(aug_json['images'][-1])
print(aug_json['annotations'][-1])



{'width': 1024, 'height': 1024, 'file_name': 'train_aug/9500.jpg', 'license': 0, 'flickr_url': None, 'coco_url': None, 'date_captured': '2020-12-26 14:44:23', 'id': 9500}
{'image_id': 9500, 'category_id': 0, 'area': 367397.36, 'bbox': [193.3, 209.2, 576.4000000000001, 637.3999999999999], 'iscrowd': 0, 'id': 39709}


In [43]:
test_result = pd.read_csv(output_dir)

file_names = test_result['image_id'].values.tolist()
bboxes = test_result['PredictionString'].values.tolist()
seudo_list=[]
    
img_list = []
ann_list = []
img_id = 9500
data_id = 39709
name_space=10000
images_frame = aug_json['images'][-1]
for idx in range(4871):
    file_name = file_names[idx]
    bbox = bboxes[idx].split()
    tmp_data=[]
    test_score = 0
    test_n = 0
    
    for i in range(0, len(bbox), 6):
        label = int(bbox[i])
        score = float(bbox[i + 1])
        x_min = float(bbox[i + 2].split('.')[0])
        y_min = float(bbox[i + 3].split('.')[0])
        x_max = float(bbox[i + 4].split('.')[0])
        y_max = float(bbox[i + 5].split('.')[0])
        if float(bbox[i+1])>0.1:
            test_score+=float(score)
            test_n+=1
            tmp_data.append([label,x_min,y_min,x_max-x_min,y_max-y_min])

    if test_n==0:
        continue
    if test_score/test_n>0.8:
        img_id+=1
        tmp_data2 = images_frame.copy()
        tmp_data2['file_name'] = tmp_data2['file_name'][:10]+str(img_id)+".jpg"
        tmp_data2['id'] = img_id
        dir=f'./dataset/test/{str(name_space+idx)[1:]}.jpg'
        tmp_img_data = np.array(Image.open(dir))
        im = Image.fromarray(tmp_img_data)
        im.save(f'./dataset/train_aug/{str(img_id)}.jpg')
        img_list.append(tmp_data2)
        for data in tmp_data:
            data_id+=1
            tmp_data3 = {
            "image_id": img_id,
            "category_id": data[0],
            "area": data[3]*data[4],
            "bbox": [data[1],data[2],data[3],data[4]],
            "iscrowd": 0,
            "id": data_id
            }
            ann_list.append(tmp_data3)

In [71]:
print(img_list[-1])
print(ann_list[14])

{'width': 1024, 'height': 1024, 'file_name': 'train_aug/10723.jpg', 'license': 0, 'flickr_url': None, 'coco_url': None, 'date_captured': '2020-12-26 14:44:23', 'id': 10723}
{'image_id': 9511, 'category_id': 7, 'area': 93852.0, 'bbox': [240.0, 479.0, 316.0, 297.0], 'iscrowd': 0, 'id': 39724}


In [72]:
aug_json['images']+=img_list.copy()
aug_json['annotations']+=ann_list.copy()

In [105]:
print(aug_json['annotations'][-25])
print(len(img_list))

{'image_id': 10703, 'category_id': 7, 'area': 128152.0, 'bbox': [538.0, 274.0, 332.0, 386.0], 'iscrowd': 0, 'id': 41391}
1223


In [73]:

with open('./dataset/train_seudo.json', 'w') as outfile:
    json.dump(aug_json, outfile)